In [1]:
import pandas
import numpy
import matplotlib.pyplot as plt
import gurobipy
from msppy.msp import MSLP
from msppy.solver import SDDP
from msppy.evaluation import EvaluationTrue

In [2]:
hydro_ = pandas.read_csv("./data/hydro.csv", index_col=0)
demand = pandas.read_csv("./data/demand.csv", index_col=0)
deficit_ = pandas.read_csv("./data/deficit.csv", index_col=0)
exchange_ub = pandas.read_csv("./data/exchange.csv", index_col=0)
exchange_cost = pandas.read_csv("./data/exchange_cost.csv", index_col=0)
thermal_ = [pandas.read_csv("./data/thermal_{}.csv".format(i), index_col=0) for i in range(4)]
hist = [pandas.read_csv("./data/hist_{}.csv".format(i), index_col=0, sep=';') for i in range(4)]
stored_initial = numpy.array(hydro_['INITIAL'][:4])
inflow_initial = numpy.array(hydro_['INITIAL'][4:8])
gamma = numpy.array(pandas.read_csv(
    "./data/gamma.csv",
    names=[0,1,2,3],
    index_col=0,
    skiprows=1,
))
sigma = [
    numpy.array(pandas.read_csv(
        "./data/sigma_{}.csv".format(i),
        names=[0,1,2,3],
        index_col=0,
        skiprows=1,
    )) for i in range(12)
]
exp_mu = numpy.array(pandas.read_csv(
    "./data/exp_mu.csv",
    names=[0,1,2,3],
    index_col=0,
    skiprows=1,
))

In [3]:
T = 3
def generator(random_state,size):
    inflow = numpy.empty([size,T,4])
    inflow[:,0,:] = inflow_initial[numpy.newaxis:,]
    for t in range(T-1):
        noise = numpy.exp(random_state.multivariate_normal(mean=[0]*4, cov=sigma[t%12],size=size))
        inflow[:,t+1,:] = noise * (
            (1-gamma[t%12]) * exp_mu[t%12]
            + gamma[t%12] * exp_mu[t%12]/exp_mu[(t-1)%12] * inflow[:,t,:]
        )
    return inflow

In [4]:
HydroThermal = MSLP(T=T, bound=0, discount=0.9906)
HydroThermal.add_Markovian_uncertainty(generator)
for t in range(T):
    m = HydroThermal[t]
    stored_now, stored_past = m.addStateVars(4, ub=hydro_['UB'][:4], name="stored")
    spill = m.addVars(4, obj=0.001, name="spill")
    hydro = m.addVars(4, ub=hydro_['UB'][-4:], name="hydro")    
    deficit = m.addVars(
        [(i,j) for i in range(4) for j in range(4)], 
        ub = [
            demand.iloc[t%12][i] * deficit_['DEPTH'][j]  
            for i in range(4) for j in range(4)
        ],
        obj = [
            deficit_['OBJ'][j] 
            for i in range(4) for j in range(4)
        ], 
        name = "deficit")
    thermal = [None] * 4
    for i in range(4):
        thermal[i] = m.addVars(
            len(thermal_[i]), 
            ub=thermal_[i]['UB'], 
            lb=thermal_[i]['LB'], 
            obj=thermal_[i]['OBJ'], 
            name="thermal_{}".format(i)
        )
    exchange = m.addVars(5,5, obj=exchange_cost.values.flatten(),
        ub=exchange_ub.values.flatten(), name="exchange")
    thermal_sum = m.addVars(4, name="thermal_sum")
    m.addConstrs(thermal_sum[i] == gurobipy.quicksum(thermal[i].values()) for i in range(4))
    for i in range(4): 
        m.addConstr(
            thermal_sum[i] 
            + gurobipy.quicksum(deficit[(i,j)] for j in range(4)) 
            + hydro[i] 
            - gurobipy.quicksum(exchange[(i,j)] for j in range(5))
            + gurobipy.quicksum(exchange[(j,i)] for j in range(5)) 
            == demand.iloc[t%12][i]
        )
    m.addConstr(
        gurobipy.quicksum(exchange[(j,4)] for j in range(5)) 
        - gurobipy.quicksum(exchange[(4,j)] for j in range(5)) 
        == 0
    )
    m.addConstrs(
        (stored_now[i] + spill[i] + hydro[i] - stored_past[i] == 0 for i in range(4)),
        uncertainty_dependent=[0,1,2,3],
    )
    if t == 0:
        m.addConstrs(stored_past[i] == stored_initial[i] for i in range(4))

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only


In [5]:
HydroThermal.discretize(
    method='RSA',
    n_Markov_states=100,
    n_sample_paths=10000,
)

In [6]:
SDDP(HydroThermal).solve(
    logFile=0,
    max_iterations=100, 
    freq_evaluations=20, 
    n_simulations=-1, 
    tol=1e-3,
)

----------------------------------------------------------------
                   SDDP Solver, Lingquan Ding                   
----------------------------------------------------------------
   Iteration               Bound               Value        Time
----------------------------------------------------------------
----------------------------------------------------------------------------
             Evaluation for approximation model, Lingquan Ding              
----------------------------------------------------------------------------
   Iteration               Bound               Value        Time
----------------------------------------------------------------------------
           1       455984.927640      1236665.506014    0.173090
           2       734669.066617       799777.544333    0.068241
           3       736134.291958       728368.474266    0.062924
           4       738289.215354       743660.149720    0.066467
           5       742393.186868       730

In [7]:
result = EvaluationTrue(HydroThermal)
result.run(n_simulations=1000, random_state=666)
result.CI

(765441.4560435136, 777711.3437271892)